In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sys
import os
import sklearn
import seaborn as sns
from sklearn.preprocessing import StandardScaler

import optuna
from sklearn.ensemble import VotingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import  OrdinalEncoder
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.preprocessing import StandardScaler

/opt/anaconda3/envs/lg_aimers/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
def data_preprocessing(train, test):
    X = train.drop(['임신 성공 여부'], axis=1)
    y = train['임신 성공 여부']

    #Data Pre-processing
    # Categorical(범주형) 칼럼 찾기
    categorical_columns = X.select_dtypes(include=['object', 'category']).columns.tolist()

    # 데이터 인코딩
    ordinal_encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)

    X_train_encoded = X.copy()
    X_train_encoded[categorical_columns] = ordinal_encoder.fit_transform(X[categorical_columns])

    X_test_encoded = test.copy()
    X_test_encoded[categorical_columns] = ordinal_encoder.transform(test[categorical_columns])

    # 불필요한 칼럼 삭제
    columns_to_drop = [
            "남성 주 불임 원인",
            "남성 부 불임 원인",
            "불임 원인 - 정자 농도",
            "불임 원인 - 정자 면역학적 요인",
            "불임 원인 - 정자 운동성",
            "불임 원인 - 정자 형태",
            '배란 유도 유형'
    ]
    X_train_encoded = X_train_encoded.drop(columns = columns_to_drop)    
    X_test_encoded = X_test_encoded.drop(columns = columns_to_drop)
    

    # 데이터 정규화 (X_train_encoded & X_test_encoded)
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train_encoded)
    X_test_scaled = scaler.transform(X_test_encoded)  # 동일한 스케일 적용


    return X_train_scaled, X_test_scaled, y


In [9]:
import sys
import os
import pandas as pd

# 현재 작업 디렉토리 경로를 가져와 shared codes 폴더의 위치를 sys.path에 추가합니다.
# sys.path에 추가된 경로에 있는 py 폴더는 임포트할 수 있다.
current_dir = os.getcwd()
shared_codes_dir = os.path.join(current_dir, '../shared codes')
sys.path.append(shared_codes_dir)


# cover_nan 모듈을 임포트
from cover_nan import missing_value_removal_function

# 원본 train 데이터 로드
train = pd.read_csv("../shared codes/data/train.csv")
test = pd.read_csv("../shared codes/data/test.csv")

# missing_value_removal_function 사용
train_df = missing_value_removal_function(train)
test_df = missing_value_removal_function(test)

X_train_scaled, X_test_scaled, y = data_preprocessing(train_df, test_df)

✅ '대리모 여부' 결측값을 최빈값 (0.0) 으로 대체 완료!
✅ 컬럼 삭제 완료: ['PGD 시술 여부', 'PGS 시술 여부', '난자 해동 경과일', '배아 해동 경과일']
✅ '난자 채취 경과일' 결측값을 중앙값 (0.0) 으로 대체 완료!
✅ '난자 혼합 경과일' 결측값을 중앙값 (0.0) 으로 대체 완료!
✅ '배아 이식 경과일' 결측값을 중앙값 (3.0) 으로 대체 완료!
✅ '대리모 여부' 결측값을 최빈값 (0.0) 으로 대체 완료!
✅ 컬럼 삭제 완료: ['PGD 시술 여부', 'PGS 시술 여부', '난자 해동 경과일', '배아 해동 경과일']
✅ '난자 채취 경과일' 결측값을 중앙값 (0.0) 으로 대체 완료!
✅ '난자 혼합 경과일' 결측값을 중앙값 (0.0) 으로 대체 완료!
✅ '배아 이식 경과일' 결측값을 중앙값 (3.0) 으로 대체 완료!


/Users/downy/Documents/2025 LG aimers/DKU-LG-Capstone-6/다훈/../shared codes/cover_nan.py:94: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['난자 출처'].replace('알 수 없음','본인 제공', inplace=True)
/Users/downy/Documents/2025 LG aimers/DKU-LG-Capstone-6/다훈/../shared codes/cover_nan.py:100: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on wh

In [11]:
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

def plot_feature_importances(X_train_scaled, y, feature_names):
    # X_train_scaled를 DataFrame으로 변환하여 컬럼 이름 설정
    X_train_df = pd.DataFrame(X_train_scaled, columns=feature_names)
    
    # Define and fit models
    models = {
        'CatBoost': CatBoostClassifier(iterations=100, depth=6, learning_rate=0.1, verbose=0),
        'XGBoost': XGBClassifier(use_label_encoder=False, eval_metric='logloss'),
        'RandomForest': RandomForestClassifier(n_estimators=100),
        'LightGBM': LGBMClassifier(n_estimators=100),
        'LogisticRegression': LogisticRegression(max_iter=200)
    }

    feature_importances = {}
    
    for name, model in models.items():
        print(f"Training {name}...")
        model.fit(X_train_df, y)
        
        if hasattr(model, 'feature_importances_'):
            importance = model.feature_importances_
        elif hasattr(model, 'coef_'):
            importance = np.abs(model.coef_[0])
        else:
            importance = np.zeros(X_train_df.shape[1])  # 기본값
        
        feature_importances[name] = pd.Series(importance, index=X_train_df.columns)

    # Plot feature importances
    for name, importances in feature_importances.items():
        importances_sorted = importances.sort_values(ascending=False)
        plt.figure(figsize=(10, 6))
        sns.barplot(x=importances_sorted, y=importances_sorted.index, palette='viridis')
        plt.title(f'Feature Importances for {name}')
        plt.xlabel('Importance Score')
        plt.ylabel('Features')
        plt.tight_layout()
        plt.show()

# Usage
# Assuming X_train_scaled is a numpy array and y is the target variable
# feature_names is a list containing the names of the features
plot_feature_importances(X_train_scaled, y, feature_names)

NameError: name 'feature_names' is not defined